# M1399.000200 Homework 1

#### Due Sep 26, 2021 @ 11:59PM

## Q1. Git/GitHub

 **No handwritten homework reports are accepted for this course.** We work     with Git and GitHub. Efficient and abundant use of Git, e.g., frequent and    well-documented commits, is an important criterion for grading your homework.

 1. Apply for the [Student Developer Pack](https://education.github.com/pack)  at GitHub using your `snu.ac.kr` email.

 1. A link to join the M1399.000200 Github Classroom and a link to create an individual Github repository for homework is provided in the eTL. First join  the classroom, and then create your own homework repo by accepting these two  invitations in turn.

 1. For each homework, the teaching assistant will make a pull request. Merge  each pull request to your homework repo.

 1. In this course, you are required to write your homework reports using IJulia. For each homework, you need to submit your Jupyter notebook (e.g, `hw1.ipynb`), html (e.g., `hw1.html`), along with all code and data that are necessary to reproduce the results.
 
 1. Provide your answer *directly* on this notebook. Add your answer right below the question. If the question has subproblems, split the cell at the right location and insert cells for your answer. **Do not modify the questions.**

 1. Maintain two branches `master` and `develop`. The `develop` branch will    be your main playground, the place where you develop solution (code) to       homework problems and write up report. The `master` branch will be your       presentation area. Submit your homework files (Jupyter notebook file `ipynb`, `html`  file converted from the notebook, all code and data sets to reproduce results)  in `master` branch.

 1. Before each homework's due date, commit your **master** branch. The        teaching assistant and the instructor will check out your committed master    branch for grading. Commit time will be used as your submission time. That    means if you commit your Homework 1 submission after the deadline, penalty    points will be deducted for late submission according to the syllabus.
 
 1. Read the [style guide](https://docs.julialang.org/en/v1/manual/style-guide/index.html) for Julia programming. Following rules in the style guide will be strictly enforced when grading: "Write functions, not just scripts", "Avoid writing overly-specific types", "Handle excess argument diversity in the caller", "Append ! to names of functions that modify their arguments",  "Don't use unnecessary static parameters", "Avoid using floats for numeric literals in generic code when possible". Also it is advised to follow "Use naming conventions consistent with Julia base/", "Write functions with argument ordering similar to Julia Base".

## Q2. Gibbs sampler in R and Julia

The task in this question is to create a Gibbs sampler for the density  
$$
f(x, y) = k x^2 exp(- x y^2 - y^2 + 2y - 4x), x > 0
$$
using the conditional distributions
$$
\begin{eqnarray*}
  X | Y &\sim& \Gamma \left( 3, \frac{1}{y^2 + 4} \right) \quad \text{(shape, scale)}\\
  Y | X &\sim& N \left(\frac{1}{1+x}, \frac{1}{2(1+x)} \right).
\end{eqnarray*}
$$

Consider the following R function `Rgibbs()`.

```r
library(Matrix)
Rgibbs <- function(N, thin) {
  mat <- matrix(0, nrow=N, ncol=2)
  x <- y <- 0
  for (i in 1:N) {
    for (j in 1:thin) {
      x <- rgamma(1, 3, y * y + 4) # 3rd arg is rate
      y <- rnorm(1, 1 / (x + 1), 1 / sqrt(2 * (x + 1)))
    }
    mat[i,] <- c(x, y)
  }
  mat
}
```

1) Explain what the above function does.

The function collects `N` random samples for the given density using Gibbs sampling technique via `thin` times of thinning.\
Actually, `Rgibbs()` function generates `N` x `thin` Gibbs samples and take only every `thin`-th sample for reducing autocorrelation between the generated Gibbs samples.

2) Write R code that generate a sample from $f$ of size 10,000 with a thinning of 500 **and** measure the time. Your R code should be a one-line. Run your code in Julia using the `RCall.jl` package.

In [98]:
using RCall
R"""
library(Matrix)
Rgibbs <- function(N, thin) {
  mat <- matrix(0, nrow=N, ncol=2)
  x <- y <- 0
  for (i in 1:N) {
    for (j in 1:thin) {
      x <- rgamma(1, 3, y * y + 4) # 3rd arg is rate
      y <- rnorm(1, 1 / (x + 1), 1 / sqrt(2 * (x + 1)))
    }
    mat[i,] <- c(x, y)
  }
  mat
}

system.time(Rgibbs(10000, 500))
"""

RObject{RealSxp}
 사용자  시스템 elapsed 
 25.874   1.505  29.077 


3) Write a Julia function `jgibbs()` that does the same compuation as `Rgibbs()`. (*Hint*. use `Distributions.jl`)

In [99]:
using Distributions

function jgibbs(N, thin)
    x = 0
    y = 0
    mat = zeros(N, 2)
    for i in 1:N, j in 1:thin
        x = rand(Gamma(3, 1 / (y * y + 4)), 1)[1] # convert 1 element Vector to scalar
        y = rand(Normal(1 / (x + 1), 1 / sqrt(2 * (x + 1))), 1)[1] # convert 1 element Vector to scalar
        mat[i, :] = [x y]
    end
    return mat
end

jgibbs (generic function with 1 method)

4) Run `jgibbs(100, 5)` and then repeat problem 2 for `jgibbs()`. How long does it take? Why do you run `jgibbs(100, 5)` before measuring the time?

I run `jgibbs(100, 5)` before measuring the time because the first run includes compilation time. \
According to the following code, it takes about 1.6 seconds.

In [100]:
jgibbs(100, 5);
@time jgibbs(10000, 500);

  2.180940 seconds (15.00 M allocations: 1.341 GiB, 20.16% gc time)


5) Compare the speed of `Rgibbs()` and `jgibbs()`. Also compare the programming efforts.

`jgibbs()` is about 20 fold faster than `Rgibbs()`. However, the programming efforts are similar due to the similar grammar.

## Q3. JIT

Consider Julia function
```julia
function g(k::Number)
    for i in 1:10
        k = 2k - 1
    end
    k
end
```
1) Use `@code_llvm` to find the LLVM bytecode of compiled `g(2)`.

In [101]:
function g(k::Number)
    for i in 1:10
        k = 2k - 1
    end
    k
end

g (generic function with 1 method)

In [102]:
@code_llvm g(2)

;  @ In[101]:1 within `g'
define i64 @julia_g_3005(i64 signext %0) {
top:
;  @ In[101]:3 within `g'
; ┌ @ int.jl:88 within `*'
   %1 = shl i64 %0, 10
; └
; ┌ @ int.jl:86 within `-'
   %2 = add i64 %1, -1023
; └
;  @ In[101]:5 within `g'
  ret i64 %2
}


2) Use `@code_llvm` to find the LLVM bytecode of compiled `g(2.0)`.

In [103]:
@code_llvm g(2.0)

;  @ In[101]:1 within `g'
define double @julia_g_3007(double %0) {
top:
;  @ In[101]:3 within `g'
; ┌ @ promotion.jl:322 within `*' @ float.jl:332
   %1 = fmul double %0, 2.000000e+00
; └
; ┌ @ promotion.jl:323 within `-' @ float.jl:329
   %2 = fadd double %1, -1.000000e+00
; └
; ┌ @ promotion.jl:322 within `*' @ float.jl:332
   %3 = fmul double %2, 2.000000e+00
; └
; ┌ @ promotion.jl:323 within `-' @ float.jl:329
   %4 = fadd double %3, -1.000000e+00
; └
; ┌ @ promotion.jl:322 within `*' @ float.jl:332
   %5 = fmul double %4, 2.000000e+00
; └
; ┌ @ promotion.jl:323 within `-' @ float.jl:329
   %6 = fadd double %5, -1.000000e+00
; └
; ┌ @ promotion.jl:322 within `*' @ float.jl:332
   %7 = fmul double %6, 2.000000e+00
; └
; ┌ @ promotion.jl:323 within `-' @ float.jl:329
   %8 = fadd double %7, -1.000000e+00
; └
; ┌ @ promotion.jl:322 within `*' @ float.jl:332
   %9 = fmul double %8, 2.000000e+00
; └
; ┌ @ promotion.jl:323 within `-' @ float.jl:329
   %10 = fadd double %9, -1.000000e+00


3) Compare the bytecode from questions 1 and 2. What do you find?  

For integer input as `g(2)` in question 1, LLVM compiler knows that the result is the same as shifting the input to the left by 10 bits and add -1023.\
On the other hand, for `Float64`(`double` in LLVM bytecode) input as `g(2.0)` in question 2, LLVM bytecode consists of iterative float-multiplication and float-addition, as the definition of `g()`.

4) Repeat questions 1-3 with `@code_native`. What is the difference from `@code_llvm`?

In [104]:
@code_native g(2)

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ In[101]:3 within `g'
; │┌ @ int.jl:88 within `*'
	shlq	$10, %rdi
; │└
; │┌ @ int.jl:86 within `-'
	leaq	-1023(%rdi), %rax
; │└
; │ @ In[101]:5 within `g'
	retq
	nopl	(%rax)
; └


1st instruction shifts the content of the general purpose 64-bit register RDI to the left by 10 bits.\
2nd instruction adds the content of the general purpose 64-bit register RDI to -1023, and load the result into another register RAX.

In [105]:
@code_native g(2.0)

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ In[101]:3 within `g'
; │┌ @ promotion.jl:322 within `*' @ float.jl:332
	vaddsd	%xmm0, %xmm0, %xmm0
	movabsq	$5157603728, %rax               ## imm = 0x1336AC990
	vmovsd	(%rax), %xmm1                   ## xmm1 = mem[0],zero
; │└
; │┌ @ promotion.jl:323 within `-' @ float.jl:329
	vaddsd	%xmm1, %xmm0, %xmm0
; │└
; │┌ @ promotion.jl:322 within `*' @ float.jl:332
	vaddsd	%xmm0, %xmm0, %xmm0
; │└
; │┌ @ promotion.jl:323 within `-' @ float.jl:329
	vaddsd	%xmm1, %xmm0, %xmm0
; │└
; │┌ @ promotion.jl:322 within `*' @ float.jl:332
	vaddsd	%xmm0, %xmm0, %xmm0
; │└
; │┌ @ promotion.jl:323 within `-' @ float.jl:329
	vaddsd	%xmm1, %xmm0, %xmm0
; │└
; │┌ @ promotion.jl:322 within `*' @ float.jl:332
	vaddsd	%xmm0, %xmm0, %xmm0
; │└
; │┌ @ promotion.jl:323 within `-' @ float.jl:329
	vaddsd	%xmm1, %xmm0, %xmm0
; │└
; │┌ @ promotion.jl:322 within `*' @ float.jl:332
	vaddsd	%xmm0, %xmm0, %xmm0
; │└
; │┌ @ promotion.jl:323 within `-' @ float.jl:329
	v

1st instruction adds the content of the 128-bit register XMM0 to itself, and overwrites the result into XMM0.\
2nd instruction moves the immediate constant $5157562920 to the 64-bit register RAX.\
3rd instruction loads the content of RAX into XMM1.\
Instructions named with `vaddsd` after the 3rd instruction add the content of the 128-bit register XMM0 to itself and overwrite the result into XMM0, and add the content of the 128-bit register XMM1 to the content of the 128-bit register XMM0 and overwrite the result into XMM0 iteratively.\
Note that the addition in this code are the floating-point arithmetic.

The difference from `@code_llvm` is that `@code_native` shows the assembly code so that it contains the detailed computation process and the location of contents.\
Especially, `@code_llvm g(2.0)` of question 2 shows the float-multiplication with `fmul`, whereas `@code_native g(2.0)` does not show the multiplication explicitly but only show the addition so that the multiplication is altered to the self-addition of the content.

## Q4. Floats

Consider the followiig model for the floating-point representation:
$$
    \pm 0.d_1d_2\dotsb d_p \times b^e
$$
with $e_{\min} \leq e \leq e_{\max}$. 

1) How many floating-point numbers are there? 

Suppose $d_1\ne 0$. Then, $$ \pm 0.d_1\dotsc d_p\times b^e = \pm d_1.d_2\dotsc d_p\times b^{e-1} $$
and in this case, there are $2(b-1)b^{p-1}(e_\max-e_\min + 1)$ different number of floating-point numbers.\
If $d_1 = 0,d_2\ne 0$, then $$ \pm 0.0d_2\dotsc d_p\times b^e = \pm d_2.d_3\dotsc d_p\times b^{e-2}.$$
If $e\ge e_\min+1$, then the floating-point numbers of the form $\pm d_2.d_3\dotsc d_p\times b^{e-2}$ are included in the case above. Thus, we only need to count the case of $e = e_\min$ and there are $2(b-1)b^{p-2}$ unique floating-point numbers in this case.\
Analogously, we can count the case of $d_1=\cdots = d_k = 0,d_{k+1}\ne 0, k<p$ as $2(b-1)b^{p-k-1}$.\
If $d_i=0$ for any $i=1,\cdots,p$, then it implies $\pm 0.0\dotsc 0\times b^e = 0$. Therefore, there are
$$
2(b-1)b^{p-1}(e_\max-e_\min+1) + 2(b-1)b^{p-2}+\cdots+2(b-1)b^0 + 1 = 2(b-1)b^{p-1}(e_\max-e_\min) + 2(b^p-1)+1
$$

2) What is the smallest positive number?

$0.0\dotsc 01\times b^{e_\min} = \frac{1}{b^p}\times b^{e_\min} = b^{e_\min - p}$

3) What is the smallest number larger than 1? 

Since $\frac{1}{b^k}\times b^k = 1$ and 
$$0.11\dotsc 11\times b^0< 1 < 0.10\dotsc 01\times b^1 < 0.010\dotsc 01\times b^2<\cdots<0.0\dotsc 011\times b^{p-1},$$
$0.10\dotsc 01\times b^1 = \left(\frac{1}{b}+\frac{1}{b^p}\right)\times b = 1+\frac{1}{b^{p-1}}$ is the smallest number larger than 1.

4) What is the smallest number $X$, such that $X + 1 = X$? Assume the round to nearest, ties to even rule.

Let $X = 0.d_1\dotsc d_p\times b^e$ and $d_1 \ne 0$. By assuming the round to nearest and ties to even rule,
$$ 2\le 0.0\dotsc 01\times b^e = b^{e-p}. $$
If above inequality is not satisfied, then $X+1\ne X$ because 1 is larger than the half of $ 0.0\dotsc 01\times b^e$ and we assumed the round to nearest rule.\
Since $b\ge 2$ in general, $e\ge p+1$. That is, the smallest number $X$ must be in the form of $0.d_1\dotsc d_{p}\times b^{p+1}$.\
Now, suppose $d_1 = 0$. Then $X = 0.d_2\dotsc d_{p}\times b^p$ and thus $X+1\ne X$ due to $2 > 0.0\dotsc 01\times b^{p} = 1$.\
Therefore, $d_1\ne 0$ and thus the smallest number of the form $0.d_1\dotsc d_{p}\times b^{p+1}$ is
$$ 
X = 0.10\dotsc 00\times b^{p+1} = b^p
$$

5) Suppose $p=4$ and $b=2$ (and $e_{\min}$ is very small and $e_{\max}$ is very large). What is the next number after 20 in this number system?

$20 = 10100_2 = 0.1010\times 2^5$. Thus, the next number after 20 is
$$ 0.1011\times 2^5 = \left(\frac{1}{2} + \frac{1}{2^3} + \frac{1}{2^4}\right)\times 2^5 = 22 $$

## Q5. Sum of squares

Typical statistical computing routines would include that for computing the sum of squares of $n$ real number:
$$
    \sum_{i=1}^n(x_i - \bar{x})^2 = \sum_{i=1}^n x_i^2 - n\bar{x}^2
$$
where $\bar{x} = \frac{1}{n}\sum_{i=1}^n x_i$.

Either expression in the above equation suggests an algoriihm. The LHS suggests 
```
# Algorithm 1
a = x[1]
for i = 2,...,n {
    a = x[i] + a 
}
a = a/n
b = (x[1] − a)^2 
for i = 2,...,n {
    b=(x[i] − a)^2 + b 
}
```

The RHS suggests
```
# Algorithm 2
a = x[1] 
b = x[1]^2
for i = 2,...,n {
    a = x[i] + a
    b = x[i]^2 + b 
}
a = a/n
b = b − n * a^2
```

1) Compare Algorithms 1 and 2. Which algoritm is more likely to end up with catastrophic cancellation?

Both Algorithm 1 and 2 computes $\bar{x}$ via $a$ with iterative addition. Algorithm 1 computes $(x_i - \bar{x})^2$ and with iterative addition, it gets the LHS. On the other hand, Algorithm 2 computes $\sum_{i=1}^n x_i^2$ first, and then by subtracting $n\bar{x}^2$, it gets the RHS. \
It seems that Algorithm 1 is more likely to end up with catastrophic cancellation. This is because of iterative evaluation of $(x_i-\bar{x})^2$; there are $n$ substraction in the algorithm and if one of $x_i$ is similar to $\bar{x}$, then the catastrophic cancellation must be occured. One can say that Algorithm 2 also may suffer the same phenomenon if $\sum_{i=1}^nx_i^2\approx n\bar{x}^2$, but this also implies that
$$
(x_i-\bar{x})^2 \le \sum_{i=1}^n(x_i - \bar{x})^2 = \sum_{i=1}^n x_i^2 - n\bar{x}^2,\quad \forall i=1,\cdots,n
$$
Thus, even if we assume $\sum_{i=1}^nx_i^2\approx n\bar{x}^2$, Algorithm 1 suffers more crucial catastrophic cancellation compared to Algorithm 2.

As an alternative, consider the following algorithm.
```
# Algorithm 3
a = x[1]
b = 0.0
for i = 2,...,n {
    d = (x[i] − a) / i 
    a = d + a 
    b = i * (i − 1) * d^2 + b
}
```

2) Show that Algorithm 3 correctly computes the sum of squares.

First, let $a_1 =x_1,b_1=0$ and 
$$
\begin{eqnarray*}
a_i &=& \frac{x_i-a_{i-1}}{i} + a_{i-1} = \frac{x_i}{i} + \frac{i-1}{i}a_{i-1}\\
b_i &=& i(i-1)\left(\frac{x_i-a_{i-1}}{i}\right)^2 + b_{i-1} = \frac{i-1}{i}(x_i-a_{i-1})^2 + b_{i-1}
\end{eqnarray*}
$$
for $i=2,\cdots,n$. Claim that 
$$
\begin{eqnarray*}
a_i &=& \frac{1}{i}\sum_{j=1}^i x_j\\
b_i &=& \sum_{j=1}^i (x_j - a_i)^2.
\end{eqnarray*}
$$
for $i=2,\cdots,n$. First, since $a_1 = x_1 = \frac{x_1}{1}$ and $ b_1 = (x_1 - a_1)^2 = 0$, the claim is satisfied for $i=1$. Suppose $a_k = \frac{1}{k}\sum_{j=1}^k x_j$ and $b_k = \sum_{j=1}^k (x_j - a_k)^2$ for $k\ge 1$. Then,
$$
\begin{eqnarray*}
a_{k+1} = \frac{x_{k+1}}{k+1} + \frac{k}{k+1}a_k &=& \frac{x_{k+1}}{k+1} + \frac{1}{k+1}\sum_{j=1}^k x_j\\
&=& \frac{1}{k+1}\sum_{j=1}^{k+1} x_j.
\end{eqnarray*}
$$
Hence, by induction, the claim for $\{a_i\}$ is proved. On the other hand, note that
$$
\begin{eqnarray*}
\frac{k}{k+1}(x_{k+1}-a_k)^2 = \frac{k}{k+1}\left(x_{k+1} - \frac{1}{k}\sum_{j=1}^k x_j\right)^2 &=& \frac{k}{k+1}\left(\frac{k+1}{k}x_{k+1} - \frac{1}{k}\sum_{j=1}^{k+1} x_j\right)^2\\
&=& \frac{k}{k+1}\left(\frac{k+1}{k}(x_{k+1} - a_{k+1})\right)^2\\
&=& \frac{k+1}{k}(x_{k+1}-a_{k+1})^2
\end{eqnarray*}
$$
and 
$$
\begin{eqnarray*}
(k+1)a_{k+1} = ka_k + x_{k+1} \Rightarrow a_k = a_{k+1} - \frac{1}{k}(x_{k+1}-a_{k+1}).
\end{eqnarray*}
$$
Thus, $b_k$ is alternatively expressed as
$$
\begin{eqnarray*}
b_k &=& \sum_{j=1}^k\left(x_j - a_{k+1} + \frac{1}{k}(x_{k+1}-a_{k+1})\right)^2\\
&=& \sum_{j=1}^k (x_j-a_{k+1})^2 + \frac{2}{k}(x_{k+1}-a_{k+1})\sum_{j=1}^k (x_j-a_{k+1}) + \frac{1}{k^2}\cdot k\cdot (x_{k+1}-a_{k+1})^2\\
&=&\sum_{j=1}^k (x_j-a_{k+1})^2 + \frac{2}{k}(x_{k+1}-a_{k+1})\left[\sum_{j=1}^{k+1} (x_j-a_{k+1})- (x_{k+1}-a_{k+1})\right] +\frac{1}{k} (x_{k+1}-a_{k+1})^2\\
&=& \sum_{j=1}^k (x_j-a_{k+1})^2 + \frac{2}{k}(x_{k+1}-a_{k+1})\left[0- (x_{k+1}-a_{k+1})\right]+\frac{1}{k} (x_{k+1}-a_{k+1})^2 \quad\left( \because \sum_{j=1}^{k+1} (x_j-a_{k+1}) = 0\right)\\
&=& \sum_{j=1}^k (x_j-a_{k+1})^2 - \frac{1}{k} (x_{k+1}-a_{k+1})^2.
\end{eqnarray*}
$$
Therefore, $b_{k+1}$ for $k\ge 1$ can be expressed as
$$
\begin{eqnarray*}
b_{k+1} &=& \frac{k+1}{k}(x_{k+1}-a_{k+1})^2+b_{k}\\
&=& \frac{k+1}{k}(x_{k+1}-a_{k+1})^2 + \sum_{j=1}^k (x_j-a_{k+1})^2 - \frac{1}{k} (x_{k+1}-a_{k+1})^2\\
&=& \sum_{j=1}^{k+1}(x_j-a_{k+1})^2
\end{eqnarray*}
$$
and by induction, the claim for $\{b_i\}$ is proved. \
Since Algorithm 3 initialize $a_1 = x_1, b_1 = 0$ and iteratively computes $a_i, b_i$ for $i=2,\cdots, n$, Algorithm 3 consequently computes $a_n$ and $b_n$, where 
$$
a_n = \frac{1}{n}\sum_{i=1}^n x_i = \bar{x},\quad b_n = \sum_{i=1}^n(x_i-a_n)^2 = \sum_{i=1}^n(x_i-\bar{x})^2.
$$

3) Compare Algorithm 3 with Algorithms 1 and 2.

Algorithm 3 sequentially computes the sample mean and sum of squares of subsequence $\{x_1,\cdots,x_i\}$ for $i = 1,\cdots, n$. That is, Algorithm 3 computes $$a_i = \frac{1}{i}\sum_{j=1}^i x_j,\quad b_i = \sum_{j=1}^i (x_j-a_i)^2$$
for each iteration, whereas Algorithms 1 and 2 do not. Since Algorithm 3 also computes $x_i - a_{i-1}$, it may suffer the catastrophic cancellation. However, $a_{i-1}$ in $x_i - a_{i-1}$ does not consider the value of $x_i$ whereas Algorithm 1 computes $x_i - \bar{x}$ where $\bar{x} = \frac{1}{n}\sum_{i=1}^nx_i$. This small difference may be a workaround for catastrophic cancellation; since the information of $x_i$ in the former approach does not contained in the subtraction explicitly. Moreover, even the catastrophic cancellation occured in Algorithm 3, we can also expect that divding with `i` in `d = (x[i] - a) / i` makes the invalid(unassigned) digits be vanished. Thus, Algorithm 3 is better than Algorithm 1 in view of catastrophic cancellation. \
However, it seems that Algorithm 3 is not robust to the catastrophic cancellation compared to Algorithm 2; re-consider the case $\sum_{i=1}^n x_i^2\approx n\bar{x}^2$. Then,
$$
 b_{n} = \sum_{i=1}^n (x_i-\bar{x})^2 = \frac{n-1}{n}\left(x_n - a_{n-1}\right)^2 + b_{n-1}\ge b_{n-1} = \sum_{i=1}^{n-1}(x_i-a_{n-1})^2
$$
and this implies $b_{i+1}\ge b_{i}$  and $x_{i+1}\approx a_{i}$ for $i=1,\cdots,n-1$. That is, even if we consider the case of catastrophic cancellation in Algorithm 2, the situation is more crucial in Algorithm 3.

4) Write a Julia function `wsos(x; w)` that computes the weighted sum of squares $\sum_{i=1^n}w_i(x_i - \bar{x})^2$ in a similar way to Algorithm 3. Here, `x` is an array of numbers, and `w` is an array of positive weights.

For $i=1,\cdots, n$, let
$$
Z_i = \sum_{j=1}^i w_j, \quad\tilde{d}_i = \frac{w_i}{Z_i} (x_i-\tilde{a}_{i-1}),
$$
and for sequences $\{\tilde{a}_i\}$ and $\{\tilde{b}_i\}$, let $\tilde{a}_1 = x_1,\tilde{b}_1 = 0$ and suppose the following is satisfied:
$$
\tilde{a}_i = \tilde{d}_i + \tilde{a}_{i-1},\quad \tilde{b}_{i} = \tilde{b}_{i-1} + \frac{Z_i\cdot Z_{i-1}}{w_i} \tilde{d}_i^2 = \tilde{b}_{i-1} + \frac{Z_{i-1}}{Z_i}w_i(x_i-\tilde{a}_{i-1})^2
$$
for $i=2,\cdots, n$.
Now, claim that for $i=1,\cdots, n$,
$$
\tilde{a}_i = \frac{1}{Z_i}\sum_{j=1}^i w_jx_j,\quad \tilde{b}_i = \sum_{j=1}^i w_j(x_j-\tilde{a}_i)^2
$$
First, since $\tilde{a}_1 = x_1,\tilde{b}_1 = 0$, the case for $i=1$ is satisfied. For general $i\ge 1$, suppose $\tilde{a}_i = \frac{1}{Z_i}\sum_{j=1}^i w_jx_j$ and $\tilde{b}_i = \sum_{j=1}^i w_j(x_j-\tilde{a}_i)^2$ are satisfied. Then, especially for $\tilde{a}_{i+1}$,
$$
\begin{eqnarray*}
\tilde{a}_{i+1} = \tilde{d}_{i+1} + \tilde{a}_i &=& \frac{w_{i+1}}{Z_{i+1}}(x_{i+1}-\tilde{a}_{i}) + \tilde{a}_i\\
&=& \frac{w_{i+1}}{Z_{i+1}}x_{i+1} + \frac{Z_i}{Z_{i+1}}\tilde{a}_i\\
&=& \frac{w_{i+1}}{Z_{i+1}}x_{i+1} + \frac{1}{Z_{i+1}}\sum_{i=1}^{i}w_jx_j\\
&=& \frac{1}{Z_{i+1}}\sum_{i=1}^{i+1}w_jx_j.
\end{eqnarray*}
$$
Hence, by induction, the claim for $\{\tilde{a}_i\}$ is proved. On the other hand, by the fact above, note that
$$
\begin{eqnarray*}
\frac{Z_{i}}{Z_{i+1}}w_{i+1}(x_{i+1}-\tilde{a}_{i})^2 = \frac{Z_{i}}{Z_{i+1}}w_{i+1}\left(x_{i+1} - \frac{1}{Z_i}\sum_{j=1}^iw_jx_j\right)^2 &=& \frac{Z_{i}}{Z_{i+1}}w_{i+1}\left(\frac{Z_{i+1}}{Z_i}x_{i+1} - \frac{1}{Z_i}\sum_{j=1}^{i+1}w_jx_j\right)^2\\
&=& \frac{Z_{i}}{Z_{i+1}}w_{i+1}\left[\frac{Z_{i+1}}{Z_i}(x_{i+1} - \tilde{a}_{i+1})\right]^2\\
&=& \frac{Z_{i+1}}{Z_i}w_{i+1}(x_{i+1} - \tilde{a}_{i+1})^2
\end{eqnarray*}
$$
and
$$
Z_{i+1}\tilde{a}_{i+1} = Z_i\tilde{a}_i + w_{i+1}x_{i+1}\Rightarrow \tilde{a}_i = \tilde{a}_{i+1} - \frac{w_{i+1}}{Z_i}(x_{i+1}-\tilde{a}_{i+1}).
$$
Thus, $\tilde{b_{i}}$ is alteratively expressed as
$$
\begin{eqnarray*}
\tilde{b_{i}} &=& \sum_{j=1}^i w_j\left[x_j-\tilde{a}_{i+1} + \frac{w_{i+1}}{Z_i}(x_{i+1}-\tilde{a}_{i+1})\right]^2\\
&=& \sum_{j=1}^i w_j(x_j-\tilde{a}_{i+1})^2 + 2\frac{w_{i+1}}{Z_i}(x_{i+1}-\tilde{a}_{i+1})\sum_{j=1}^i w_j(x_j-\tilde{a}_{i+1}) + \frac{w_{i+1}^2}{Z_i^2}(x_{i+1}-\tilde{a}_{i+1})^2\sum_{j=1}^iw_j\\
&=& \sum_{j=1}^i w_j(x_j-\tilde{a}_{i+1})^2 + 2\frac{w_{i+1}}{Z_i}(x_{i+1}-\tilde{a}_{i+1})\left[\sum_{j=1}^{i+1}w_j(x_j-\tilde{a}_{i+1}) - w_{i+1}(x_{i+1}-\tilde{a}_{i+1})\right] + \frac{w_{i+1}^2}{Z_i}(x_{i+1}-\tilde{a}_{i+1})^2\\
&=& \sum_{j=1}^i w_j(x_j-\tilde{a}_{i+1})^2 - \frac{w_{i+1}^2}{Z_i}(x_{i+1}-\tilde{a}_{i+1})^2\quad \left( \because \sum_{j=1}^{i+1}w_j(x_j-\tilde{a}_{i+1}) = 0\right)
\end{eqnarray*}
$$
Therefore, $\tilde{b}_{i+1}$ for $i\ge 1$ can be expressed as
$$
\begin{eqnarray*}
\tilde{b}_{i+1} &=& \tilde{b}_{i} + \frac{Z_{i+1}}{Z_i}w_{i+1}(x_{i+1} - \tilde{a}_{i+1})^2\\
&=& \sum_{j=1}^i w_j(x_j-\tilde{a}_{i+1})^2 - \frac{w_{i+1}^2}{Z_i}(x_{i+1}-\tilde{a}_{i+1})^2 + \frac{Z_{i} + w_{i+1}}{Z_i}w_{i+1}(x_{i+1} - \tilde{a}_{i+1})^2\\
&=& \sum_{j=1}^i w_j(x_j-\tilde{a}_{i+1})^2 + w_{i+1}(x_{i+1} - \tilde{a}_{i+1})^2 = \sum_{j=1}^{i+1} w_j(x_j-\tilde{a}_{i+1})^2
\end{eqnarray*}
$$
and by induction, the claim for $\{\tilde{b}_i\}$ is proved. Now, for $i=1,\cdots, n$, let $b_i = \sum_{j=1}^iw_j(x_j-a_i)^2$ where $a_i$ is defined in question 2. Then, following is satisfied:
$$
\begin{eqnarray*}
b_i - \tilde{b}_i &=& \sum_{j=1}^i w_j[(x_j-a_i)^2-(x_j-\tilde{a}_i)^2]\\
&=& \sum_{j=1}^i w_j[a_i^2 - 2a_ix_j + 2\tilde{a}_ix_j - \tilde{a}_i^2]\\
&=& Z_i(a_i^2-\tilde{a}_i^2) + 2(\tilde{a}_i - a_i)\sum_{j=1}^i w_jx_j\\
&=& Z_i(a_i-\tilde{a}_i)^2 \quad\left( \because \sum_{j=1}^i w_jx_j = Z_i\tilde{a}_i\right)
\end{eqnarray*}
$$
Therefore, $b_n = \sum_{i=1}^n w_i(x_i-\bar{x})^2 = \tilde{b}_n + Z_n(a_n-\tilde{a}_n)^2$ and the following function illustrates the process we have shown above.

In [107]:
function wsos(x; w = ones(length(x))) # default weight : w1 = ... = wn = 1
    a = x[1]      # a_1 = x_1
    ã = x[1]      # \tilde{a}_1 = x_1
    Z = w[1]      # Z_1 = w_1
    b = 0.0       # b_1 = 0
    b̃ = 0.0       # \tilde{b}_1 = 0
    n = length(x)
    for i in 2:n
        Z = Z + w[i]                               # Z_i = Z_{i-1} + w_i
        d = (x[i] - a) / i                         # d_i = (x_i - a_{i-1}) / i
        d̃ = w[i] * (x[i] - ã) / Z                  # \tilde{d}_i = w_i * (x_i - \tilde{a}_{i-1}) / Z_i
        a = d + a                                  # a_i = d_i + a_{i-1}
        ã = d̃ + ã                                  # \tilde{a}_i = \tilde{d}_i + \tilde{a}_{i-1}
        b̃ = b̃ + Z * (Z / w[i] - 1) * d̃^2          # \tilde{b}_i = \tilde{b}_{i-1} + Z_i * Z_{i-1} * \tilde{d}_i^2 / w_i
        b = b̃ + Z * (a - ã)^2                     # b_i = \tilde{b}_i + Z_i * (a_i - \tilde{a}_i)^2
    end
    return b
end

wsos (generic function with 1 method)